In [ ]:
# source: https://github.com/microsoft/autogen/blob/main/notebook/agentchat_groupchat_research.ipynb

# !pip install pyautogen
# !pip install openai
# or pip install -r requirements.txt (from the github repo)

In [ ]:
from autogen import config_list_from_json

config_list_gpt4 = config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

In [ ]:
gpt4_config = {
    "cache_seed": 42,
    "temperature": 0.0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

In [ ]:
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager

In [ ]:
user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss\
        the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False
)

In [ ]:
engineer = AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. Wrap the code in a code block that specifies the script type. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
""",
)

In [ ]:
scientist = AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan.
    You are able to categorize papers after seeing their abstracts printed.
    You don't write code.""",
)

In [ ]:
planner = AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback
    from admin and critic, until admin approval. The plan may involve an engineer
    who can write code and a scientist who doesn't write code. Explain the plan 
    first. Be clear which step is performed by an engineer, and which step is 
    performed by a scientist.
    """,
    llm_config=gpt4_config
)